In [1]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras import optimizers
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import Sequential, layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import regularizers, Sequential, layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.constraints import unit_norm
import numpy as np
from tqdm import tqdm
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import random

Init Plugin
Init Graph Optimizer
Init Kernel


In [83]:
df = pd.read_csv('../speech_emotion_reco/data/merged_dataset.csv')
df_train = pd.read_csv('df_train.csv')
df_test = pd.read_csv('df_test.csv')

In [129]:
def load_data():
    data_path = '../speech_emotion_reco/data/images_augmented/'
    classes = {'happy': 0, 
          'sad': 1,
          'fear': 2,
          'disgust': 3,
          'angry': 4,
          'neutral': 5,
          'surprise': 6,
          'unknown': 7}
    
    imgs_train = []
    imgs_test = []
    labels_train = []
    labels_test = []
    test_path = []
    
    images_path = [os.path.join(data_path, path) for path in os.listdir(data_path) if path.find('.png') > 0]
    for path in images_path:
        if path in df_train.path.values:
            index = int(path.replace('.png','').replace(data_path,'').split('_')[1])
            image = Image.open(path).convert('RGB')
            resized_image = image.resize((256,256))
            imgs_train.append(np.array(resized_image))
            labels_train.append(classes[df.loc[index, 'emotion']])
        if path in df_test.path.values:
            image = Image.open(path).convert('RGB')
            resized_image = image.resize((256,256))
            imgs_test.append(np.array(resized_image))
            labels_test.append(classes[df.loc[index, 'emotion']])
            test_path.append(path)

    X_train = np.array(imgs_train)
    X_test = np.array(imgs_test)
    y_train = np.array(labels_train)
    y_test = np.array(labels_test)
    
    return X_train, X_test, y_train, y_test, test_path

In [ ]:
X_train, X_test, y_train, y_test, test_path = load_data()

In [111]:
y = np.hstack((y_train, y_test))
y = to_categorical(y,  num_classes = 6)
y_train = y[:40045] 
y_test = y[40045:]

In [11]:
def load_model():
    model = Sequential()
    model.add(layers.Conv2D(120, kernel_size=11, activation="relu", strides=4, input_shape=(256,256,3),kernel_regularizer=regularizers.l2(l=0.01)))
    model.add(layers.MaxPooling2D(3, strides=2))
    model.add(layers.Conv2D(256, kernel_size=5, activation="relu", strides=1, kernel_regularizer=regularizers.l2(l=0.01)))
    model.add(layers.MaxPooling2D(3, strides=2))
    model.add(layers.Conv2D(384, kernel_size=3, activation="relu", kernel_regularizer=regularizers.l2(l=0.01)))
    model.add(layers.MaxPooling2D(3, strides=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(2048, activation='relu', kernel_regularizer=regularizers.l2(l=0.01)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(2048, activation='relu', kernel_regularizer=regularizers.l2(l=0.01)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(6, activation='softmax'))
    
    return model

In [12]:
def build_model():  
    model = load_model()
    model.compile(loss='categorical_crossentropy',
                  optimizer ='adam',
                  metrics=['accuracy'])
    return model

In [13]:
from tensorflow.keras.callbacks import EarlyStopping

model = build_model()

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train, 
                    validation_split=0.2,
                    epochs=100, 
                    batch_size=100,
                    callbacks=[es],
                    verbose=1)

Metal device set to: Apple M1


2021-08-31 08:44:31.891073: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-31 08:44:32.053986: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-08-31 08:44:32.071320: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2021-08-31 08:44:34.178347: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (regis

Epoch 1/100


2021-08-31 08:44:35.452816: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-08-31 08:48:49.294968: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


321/321 - 275s - loss: 101.5489 - accuracy: 0.1810 - val_loss: 58.9481 - val_accuracy: 0.1722
Epoch 2/100
321/321 - 309s - loss: 51.8738 - accuracy: 0.1948 - val_loss: 45.6934 - val_accuracy: 0.2462
Epoch 3/100
321/321 - 324s - loss: 40.9930 - accuracy: 0.2441 - val_loss: 36.6927 - val_accuracy: 0.2672
Epoch 4/100
321/321 - 329s - loss: 33.2297 - accuracy: 0.2664 - val_loss: 29.9943 - val_accuracy: 0.2978
Epoch 5/100
321/321 - 337s - loss: 27.3408 - accuracy: 0.3260 - val_loss: 24.7201 - val_accuracy: 0.3877
Epoch 6/100
321/321 - 354s - loss: 22.5487 - accuracy: 0.4155 - val_loss: 20.5217 - val_accuracy: 0.4228
Epoch 7/100
321/321 - 353s - loss: 18.7401 - accuracy: 0.4527 - val_loss: 17.0759 - val_accuracy: 0.4626
Epoch 8/100
321/321 - 358s - loss: 15.6444 - accuracy: 0.4852 - val_loss: 14.2711 - val_accuracy: 0.5057
Epoch 9/100
321/321 - 361s - loss: 13.1138 - accuracy: 0.5103 - val_loss: 11.9810 - val_accuracy: 0.5224
Epoch 10/100
321/321 - 342s - loss: 11.0301 - accuracy: 0.5411 - v

...

321/321 - 430s - loss: 2.3851 - accuracy: 0.6417 - val_loss: 2.2443 - val_accuracy: 0.6387
Epoch 21/100


Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (IOAF code 14)
	<AGXM1FamilyCommandBuffer: 0x7fd7d9a32770>
    label = <none> 
    device = <AGXG13GDevice: 0x7fddcb6ae400>
        name = Apple M1 
    commandQueue = <AGXM1FamilyCommandQueue: 0x7fdd9f237400>
        label = <none> 
        device = <AGXG13GDevice: 0x7fddcb6ae400>
            name = Apple M1 
    retainedReferences = 1
Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (IOAF code 14)
	<AGXM1FamilyCommandBuffer: 0x7fd7d9a4ad40>
    label = <none> 
    device = <AGXG13GDevice: 0x7fddcb6ae400>
        name = Apple M1 
    commandQueue = <AGXM1FamilyCommandQueue: 0x7fdd9f237400>
        label = <none> 
        device = <AGXG13GDevice: 0x7fddcb6ae400>
            name = Apple M1 
    retainedReferences = 1


321/321 - 422s - loss: 2.1201 - accuracy: 0.6468 - val_loss: 2.0285 - val_accuracy: 0.6414
Epoch 22/100


Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (IOAF code 14)
	<AGXM1FamilyCommandBuffer: 0x7fd7d9c32aa0>
    label = <none> 
    device = <AGXG13GDevice: 0x7fddcb6ae400>
        name = Apple M1 
    commandQueue = <AGXM1FamilyCommandQueue: 0x7fdd9f237400>
        label = <none> 
        device = <AGXG13GDevice: 0x7fddcb6ae400>
            name = Apple M1 
    retainedReferences = 1


321/321 - 796s - loss: 1.9261 - accuracy: 0.6501 - val_loss: 1.8813 - val_accuracy: 0.6350
Epoch 23/100
321/321 - 436s - loss: 1.7578 - accuracy: 0.6573 - val_loss: 1.7276 - val_accuracy: 0.6449
Epoch 24/100
321/321 - 309s - loss: 1.6538 - accuracy: 0.6590 - val_loss: 1.5957 - val_accuracy: 0.6555
Epoch 25/100
321/321 - 290s - loss: 1.5520 - accuracy: 0.6620 - val_loss: 1.6277 - val_accuracy: 0.6167
Epoch 26/100
321/321 - 312s - loss: 1.4845 - accuracy: 0.6639 - val_loss: 1.4465 - val_accuracy: 0.6674
Epoch 27/100
321/321 - 316s - loss: 1.4176 - accuracy: 0.6683 - val_loss: 1.4521 - val_accuracy: 0.6377
Epoch 28/100
321/321 - 319s - loss: 1.3838 - accuracy: 0.6639 - val_loss: 1.4203 - val_accuracy: 0.6427
Epoch 29/100
321/321 - 319s - loss: 1.3463 - accuracy: 0.6661 - val_loss: 1.3524 - val_accuracy: 0.6573
Epoch 30/100
321/321 - 317s - loss: 1.3276 - accuracy: 0.6674 - val_loss: 1.3353 - val_accuracy: 0.6583
Epoch 31/100
321/321 - 316s - loss: 1.2918 - accuracy: 0.6729 - val_loss: 1.4